In [ ]:
# download libraries

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.data import Dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# 1. Download data and visualisation

In [ ]:
# path to the data

hole_path = "/kaggle/input/images-of-galaxies-and-black-holes/BlackHole/BlackHole"
galaxy_path = "/kaggle/input/images-of-galaxies-and-black-holes/Galaxy/Galaxy"

In [ ]:
# function which gives the list of files names in mentioned folder

def get_jpg_filenames(folder_path):
    jpg_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".jpg")]
    return jpg_files

In [ ]:
hole_files = get_jpg_filenames(hole_path)

galaxy_files = get_jpg_filenames(galaxy_path)

In [ ]:
hole_lables = [0] * len(hole_files)

galaxy_lables = [1] * len(galaxy_files)

In [ ]:
# Dataframes with names of files and its labeles

df_hole = pd.DataFrame({"file_name": hole_files, "label": hole_lables})

In [ ]:
df_galaxy = pd.DataFrame({"file_name": galaxy_files, "label": galaxy_lables})

In [ ]:
df = pd.concat([df_hole, df_galaxy], ignore_index=True)

# For the binary classification labels should be string type

df['label'] = df['label'].astype(str)

df.sample(5)

## 1.1 Verification of class distribution 

In [ ]:
df['label'].hist(bins=5, figsize=(6, 3))

# Axies
plt.xlabel('Classies')
plt.ylabel('Entries')
plt.title('Class distribution')

plt.show()

## 1.2 Creating of training and tests dataset 

In [ ]:
# Split into training and testing sets

train_files, test_files, train_labels, test_labels = train_test_split(
    df['file_name'], df['label'], test_size=0.2, random_state=42, stratify=df['label'])

In [ ]:
df_train = pd.DataFrame({"file_name": train_files, "label": train_labels})

df_train.sample(5)

In [ ]:
df_test = pd.DataFrame({"file_name": test_files, "label": test_labels})

df_test.sample(5)

## 1.3 Image generator

In [ ]:
# Create an image generator for training with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Create training and testing generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory="file_name/kaggle/input/images-of-galaxies-and-black-holes",  # root derictory
    x_col="file_name",  # column with file names
    y_col="label",  # column with lables
    target_size=(244, 244),
    batch_size=32,
    class_mode="binary",  # mode of classification
    seed=42,
    subset='training'
)

In [ ]:
# Create an image generator for testing (without data augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory="file_name/kaggle/input/images-of-galaxies-and-black-holes",  # root derictory
    x_col="file_name",  # column with file names
    y_col="label",  # column with lables
    target_size=(244, 244),
    batch_size=32,
    class_mode="binary",  # mode of classification
    seed=42
)

## 1.4 Visualisation of datasets

In [ ]:
# visualisation of datasets
features, target = next(train_generator)

# plot 32 images
fig = plt.figure(figsize=(12,12))
for i in range(32):
    fig.add_subplot(4, 8, i+1)
    plt.imshow(features[i])
    plt.title(f'{target[i]}')
	# remove axies
    plt.xticks([])
    plt.yticks([])
    plt.suptitle('Images with labels',  y=0.9,fontsize=16, color='b')
    plt.tight_layout()

# 2. Model creation

Сode below creates an instance of the Xception model, which is a deep convolutional neural network architecture designed for image classification. Here's a breakdown of the parameters:

weights='imagenet': This specifies that you want to load pre-trained weights from the ImageNet dataset. ImageNet is a large dataset with millions of labeled images used for training deep neural networks.

input_shape=(244, 244, 3): This sets the expected shape of input images that the model will process. In this case, it expects images with a height and width of 180 pixels and three color channels (RGB).

include_top=False: This parameter specifies that you do not want to include the top layers of the model. The top layers usually consist of densely connected layers responsible for classifying objects into specific classes based on the ImageNet labels. By setting include_top to False, you are excluding these classification layers, which is often done when you want to use the model for feature extraction or as part of a custom neural network.

In summary, this code creates the base of the Xception model with pre-trained weights, configured to accept input images of size (244, 244, 3), and without the final classification layers. You can then add your own layers on top of this base model to adapt it to your specific task, like fine-tuning it for a different classification problem or using it for feature extraction.

In [ ]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(244, 244, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [ ]:
base_model.trainable = False

inputs = keras.Input(shape=(244, 244, 3)): This line defines the input layer of your model. It specifies that your model will expect input images of shape (244, 244, 3), where 244 is the height and width, and 3 is the number of color channels (RGB).

x = base_model(inputs): This line connects the previously defined base_model (Xception in this case) to the input layer. It effectively sets up a pipeline where the input images will be processed by the pre-trained Xception model.

x = keras.layers.GlobalAveragePooling2D()(x): After passing through the base model, the features are processed by a Global Average Pooling 2D layer. This layer computes the average value of each feature map in the spatial dimensions, resulting in a fixed-size vector regardless of the input size. This is often used to reduce the spatial dimensions and flatten the features before passing them to the final classification layers.

outputs = keras.layers.Dense(1, activation='sigmoid')(x): This line adds a Dense layer with a single unit and a sigmoid activation function. This is the final layer responsible for binary classification. The output is a single value between 0 and 1, representing the probability of the input image belonging to the positive class (e.g., in binary classification, 1 represents the positive class).

model = keras.Model(inputs, outputs): Finally, this line creates the overall model by specifying the inputs and outputs. This is a common way to define a model in Keras using the Functional API. The resulting model can be trained using appropriate data and optimization techniques.

In summary, this code constructs a neural network model for binary classification using a pre-trained Xception base model. The Global Average Pooling layer is used to reduce spatial dimensions, and a Dense layer with a sigmoid activation function produces the final classification output.

In [ ]:
inputs = keras.Input(shape=(244, 244, 3))
x = base_model(inputs)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

epochs = 10: This line sets the number of epochs, which is the number of times the entire dataset will be passed forward and backward through the neural network during training.

callbacks: This is a list of callback functions that will be called during training. Two common callbacks are used here:

ModelCheckpoint: This callback saves the model's weights at different points during training. In this case, it will save the weights after each epoch in a file named "save_at_{epoch}.keras".

EarlyStopping: This callback stops training when a monitored metric (in this case, validation loss) has stopped improving. The patience parameter defines the number of epochs with no improvement after which training will be stopped.

model.compile(...): This line compiles the model, specifying the optimizer, loss function, and metrics to be used during training.

optimizer=keras.optimizers.Adam(1e-3): The Adam optimizer is used with a learning rate of 1e-3 (0.001).

loss="binary_crossentropy": This is the loss function used for binary classification tasks.

metrics=["accuracy"]: During training, accuracy will be monitored.

history = model.fit(...): This line trains the model using the specified training data (train_generator) and validation data (test_generator). The training process is run for the specified number of epochs, and the specified callbacks are applied during training.

train_generator: The generator for training data.

epochs=epochs: The number of epochs to train the model.

callbacks=callbacks: The list of callbacks to be applied during training.

validation_data=test_generator: The generator for validation data.

The training history, including metrics like loss and accuracy over epochs, is stored in the history variable. This information can be used for analysis and visualization.

In [ ]:
epochs = 5

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"), keras.callbacks.EarlyStopping(patience=5)
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

history = model.fit(
    train_generator,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=test_generator,
)

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.summary()

In [ ]:
epochs = 5

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"), keras.callbacks.EarlyStopping(patience=5)
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

history = model.fit(
    train_generator,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=test_generator,
)

# 3. Test model

In [ ]:
model.evaluate(test_generator)

In [ ]:
def visualisation_of_results(generator):
    """
    Images in batch should be 32 for correct visualisation of figures
    """
    # Visualisation of datasets
    features, target = next(generator)
    # Get model predictions for batch
    preds_proba = model.predict(features)
    # Create a grid of images
    fig, axes = plt.subplots(4, 8, figsize=(22, 11))

    # Display each image in the grid
    for i, ax in enumerate(axes.flatten()):
        ax.imshow(features[i])
        ax.set_title(f'Prediction: {np.round(preds_proba[i], 2)}')
        ax.axis('off')
    
    plt.show()

In [ ]:
visualisation_of_results(test_generator)

**Even with frozen base model we reached accuracy on test model equal to 0.95. If we unfoze model, after training it on additional dataset we reach accuracy 0.975**